In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xlrd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sio
import matplotlib
import pylab as pl

In [ ]:
# Get raw frequency vector for all words at given t
def log_freq_t(t,freqs):
    if t >= 1800:
        freq_list = list(freqs.ix[:,t-1800])
        print len(freq_list)
        return freq_list
    else:
        return list(np.log(np.add(list(freqs.ix[:,t-1500]),1)))

In [ ]:
# Read in start date info
def get_start_dates(filename,st):
    to_include = []
    workbook = xlrd.open_workbook(filename)
    sheet = workbook.sheet_by_index(0)
    start_dates = {}
    for i in range(1,sheet.nrows):
        row = sheet.row_values(i)
        word = row[0]
        if word == False:   # excel auto-formats 'true','false' to keywords
            word = 'false'
        elif word == True:
            word = 'true'
        starts = row[1:len(row)]
        starts = [x for x in starts if isinstance(x,float) or x == 'OE']
        start_dates[word]=sorted(starts)
    if st=='dates':
        return start_dates

In [ ]:
# Read in end date info. -1 means the sense remains in use currently.
def get_end_dates(filename):
    workbook = xlrd.open_workbook(filename)
    sheet = workbook.sheet_by_index(0)
    end_dates = {}
    for i in range(1,sheet.nrows):
        row = sheet.row_values(i)
        word = row[0]
        if word == False:    # excel auto-formats 'true','false' to keywords
            word = 'false'
        elif word == True:
            word = 'true'
        ends = row[1:len(row)]
        ends = [x for x in ends if isinstance(x,float)]
        end_dates[word]=sorted(ends)
    return end_dates

In [ ]:
def regress_for_t(t,years,betas_f,pvals_f,betas_s,pvals_s,betas_l,pvals_l,words):
    variables=vectors_at_t(t)
    Y = variables['hte']
    
    if np.any(Y) != 0: #there are words changing
        log_X_f = np.log(variables['freq'])
        X_l = variables['lengths']
        log_X_s = np.log(np.add(variables['polysemy'],1))
        X = np.vstack((log_X_f,X_l,log_X_s))
        X = np.transpose(X)

        if len(variables['words']) > 1:
            if ~np.isnan(X[0][0]):    
                md = statsmodels.regression.mixed_linear_model.MixedLM(Y, X, variables['words']) 
                mdf = md.fit() 
                years.append(t)
                mdf_params = list(mdf.params)
                mdf_pvalues = list(mdf.pvalues)
                betas_l.append(mdf_params[1])
                pvals_l.append(mdf_pvalues[1])
                betas_s.append(mdf_params[2])
                pvals_s.append(mdf_pvalues[2])
                betas_f.append(mdf_params[0])
                pvals_f.append(mdf_pvalues[0])
                if np.isnan(mdf_params[0]):
                    print 'Y:',Y
                    print 'log Xf:',log_X_f

In [ ]:
def num_senses_prior(t,words,startdates,enddates):
    senses_prior_per_word = {}
    for w in words:
        starts = startdates[w]
        ends = enddates[w]
        starts = [s for s in starts if s < t and s > 0 or s == 'OE']
        ends = [e for e in ends if e > 0 and e < t or e == 'OE']
        senses_prior_per_word[w]=len(starts)-len(ends)
    return senses_prior_per_word

In [ ]:
def num_senses_gained(t1,t2,words,startdates):
    senses_gained_per_word = {}
    for w in words:
        starts = startdates[w]
        emerging = [d for d in starts if d >= t1 and d < t2 or d == 'OE']
        senses_gained_per_word[w]=len(emerging)
    return senses_gained_per_word

In [ ]:
def num_senses_lost(t1,t2,words,enddates):
    senses_lost_per_word = {}
    for w in words:
        ends = enddates[w]
        disappearing = [d for d in ends if d >= t1 and d < t2 or d == 'OE']
        senses_lost_per_word[w]=len(disappearing)
    return senses_lost_per_word

In [ ]:
def vectors_at_t(t,words,start_dates,end_dates):
    rate_t=[]
    freq_t=[]
    words_t=[]
    lengths_t=[]
    polysemy_t=[]
    gain_t = []
    prior_senses_t = num_senses_prior(t,words,start_dates,end_dates)
    hte_t = num_senses_gained(t-10,t,words,start_dates)
    for w in words:
        if t in rate_data[w].keys() and t in freq_ham[w].keys():
            rate = rate_data[w][t]
            freq = freq_ham[w][t]
            poly = prior_senses_t[w]
            hte = hte_t[w]
            if not np.isnan(freq) and not np.isnan(rate):
                rate_t.append(rate_data[w][t])
                freq_t.append(freq_ham[w][t])
                words_t.append(w)
                lengths_t.append(len(w))
                polysemy_t.append(poly)
                gain_t.append(hte/10.0)
    return {'freq':freq_t,'words':words_t,'lengths':lengths_t,'polysemy':polysemy_t,'rate':rate_t,'hte':gain_t}

In [ ]:
def regress_ham(words):
    
    betas_f = []
    pvals_f = []
    betas_s = []
    pvals_s = []
    betas_l = []
    pvals_l = []
    years = []
    for t in range(1810,1990,10):
        print "start of regression"
        regress_for_t(t,years,betas_f,pvals_f,betas_s,pvals_s,betas_l,pvals_l,words)
    xs = np.hstack((np.hstack((betas_f[-len(years):],betas_l[-len(years):])),betas_s[-len(years):]))
    ys = np.hstack((np.hstack((['Frequency']*len(years),['Length']*len(years))),['#Senses']*len(years)))
    to_plot = [ys,xs]
    col_labels = ['Predictor','Beta']
    df = pd.DataFrame.transpose(pd.DataFrame(to_plot, col_labels))
    %matplotlib inline
    sns.barplot(x='Beta', y='Predictor', data=df)
    sns.despine(top=True, right=True)
    plt.show()
        
    print "mean beta (log freq):", np.average(betas_f[-len(years):])
    print "mean beta (raw length):",np.average(betas_l[-len(years):])
    print "mean beta (log #prior senses):",np.average(betas_s[-len(years):])
        
    # Print #significant p-values applying Bonferroni correction
    sig_l = [x for x in pvals_l if x <= 0.05/len(pvals_l)]
    sig_f = [x for x in pvals_f if x <= 0.05/len(pvals_l)]
    sig_s = [x for x in pvals_s if x <= 0.05/len(pvals_l)]
    print '#significant p-values, frequency:',len(sig_f)
    print '#significant p-values, length:',len(sig_l)
    print '#significant p-values, #prior senses:',len(sig_s)
    print '#total regressions:',len(pvals_l)
        
    t_stats = [stats.ttest_ind(betas_s,betas_f)[0],stats.ttest_ind(betas_s,betas_l)[0],stats.ttest_ind(betas_f,betas_l)[0]]
    p_vals = [stats.ttest_ind(betas_s,betas_f)[1],stats.ttest_ind(betas_s,betas_l)[1],stats.ttest_ind(betas_f,betas_l)[1]]
    #t_stats = [stats.ttest_ind(betas_f,betas_l)[0]]
    #p_vals = [stats.ttest_ind(betas_f,betas_l)[1]]
    data = [t_stats,p_vals]
    row_labels = row_labels = ['t-stat','p-value']
    col_labels = ['Beta (S) vs. Beta (F)','Beta (S) vs. Beta (L)','Beta (F) vs. Beta (L)']
    #['Beta (F) vs. Beta (L)']
    print '\n'
    print pd.DataFrame(data, row_labels, col_labels)
        
    %matplotlib inline
    plt.title('p-values over time (length)')
    plt.xlabel('Year')
    plt.ylabel('Log p-value')
    plt.scatter(years,np.log10(pvals_l))
    plt.show()
        
    %matplotlib inline
    plt.title('p-values over time (frequency)')
    plt.xlabel('Year')
    plt.ylabel('Log p-value')
    plt.scatter(years,np.log10(pvals_f))
    plt.show()
    
    %matplotlib inline
    plt.title('p-values over time (polysemy)')
    plt.xlabel('Year')
    plt.ylabel('Log p-value')
    plt.scatter(years,np.log10(pvals_s))
    plt.show()